In [3]:
pip list

Package                                  Version             Editable project location
---------------------------------------- ------------------- -------------------------
absl-py                                  1.4.0
accelerate                               0.33.0
access                                   1.1.9
affine                                   2.4.0
aiobotocore                              2.13.2
aiofiles                                 22.1.0
aiohttp                                  3.9.5
aioitertools                             0.11.0
aiosignal                                1.3.1
aiosqlite                                0.20.0
albucore                                 0.0.13
albumentations                           1.4.14
alembic                                  1.13.2
altair                                   5.4.0
annotated-types                          0.7.0
annoy                                    1.17.3
ansicolors                               1.1.8
anyio             

In [4]:
!pip install accelerate -U
#!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 2.3 MB/s eta 0:00:00a 0:00:01m
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [2]:
pip install nltk==3.8.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!git clone https://github.com/ryanzhumich/AESLC

Cloning into 'AESLC'...
remote: Enumerating objects: 17469, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 17469 (delta 1), reused 0 (delta 0), pack-reused 17461 (from 1)
Receiving objects: 100% (17469/17469), 7.36 MiB | 15.70 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [6]:
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and word.isalnum()]

    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [8]:
# Custom dataset class
class EmailSubjectDataset(Dataset):
    def __init__(self, emails, subjects, tokenizer, max_length):
        self.emails = emails
        self.subjects = subjects
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.emails)

    def __getitem__(self, idx):
        email_text = self.emails[idx]
        subject_text = self.subjects[idx]

        # Tokenize inputs and outputs
        inputs = self.tokenizer(email_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                padding='max_length')
        outputs = self.tokenizer(subject_text, return_tensors='pt', max_length=self.max_length, truncation=True,
                                 padding='max_length')

        return {
            'input_ids': inputs.input_ids.flatten(),
            'attention_mask': inputs.attention_mask.flatten(),
            'labels': outputs.input_ids.flatten()
        }


In [9]:
# Directory containing email files
emails_directory = './AESLC/enron_subject_line/train'


In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from safetensors.torch import load_file as load_safetensors

In [ ]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 

The secret `HF_TOKEN` does not exist in your Colab secrets.

To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.

You will be able to reuse this secret in all of your notebooks.

Please note that authentication is recommended but still optional to access public models or datasets.

  warnings.warn(


In [13]:
# Load emails and subjects from files
emails = []
subjects = []

for filename in os.listdir(emails_directory):

    with open(os.path.join(emails_directory, filename), 'r', encoding='utf-8', errors='ignore') as file:
        email_text = file.read()
        # Extract subject from email body using regex
        match = re.search(r'@subject\s+(.*)', email_text, re.IGNORECASE | re.DOTALL)

        if match:
            subject = match.group(1).strip()
            email_body = re.sub(r'@subject\n(.+)\n', '', email_text).strip()
            preprocessed_email = preprocess_text(email_body)

            emails.append(preprocessed_email)
            subjects.append(subject)

In [15]:
dataset = EmailSubjectDataset(emails, subjects, tokenizer, max_length=512)

In [18]:
import os
import re
import pandas as pd

In [20]:
def re_subject(subject):
    return re.sub(r'[^a-zA-Z0-9\s]', '', subject)

In [25]:
import numpy as np

In [68]:
# Function to generate subject lines from email text
def get_words_split(text):
    words = text.split()
    return ' '.join(words[:4])

In [69]:
def generate_subject(model, tokenizer, email_text, max_length=512):
    # Move model to correct device
    device = next(model.parameters()).device

    email_text = preprocess_text(email_text)

    input_ids = tokenizer.encode(email_text, return_tensors='pt').to(device)  # Move input_ids to device

    # Generate subject line
    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_beams=5, early_stopping=True)

    generated_subject = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_subject

In [ ]:
model_path = "/kaggle/input/gpt2/transformers/gpt21/1"
state_dict = load_safetensors(f"{model_path}/model.safetensors")
model = GPT2LMHeadModel.from_pretrained(model_path, state_dict=state_dict)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [35]:
!pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=782665e6d590f8372750aff08a719b2fdbabf596c93864a5da73ef545e7609cb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [36]:
from rouge_score import rouge_scorer, scoring

In [71]:
import re


In [83]:
# Define a function to extract annotations
def extract_annotations(email_body):
    annotations = {}
    final_annotations = []
    current_annotation = None

    # Split the email body by lines
    lines = email_body.strip().split('\n')

    # Iterate through each line
    for line in lines:
        line = line.strip()

    
        if line.startswith('@ann'):
            current_annotation = line
            annotations[current_annotation] = []
        elif current_annotation:
            # Append line to current annotation's text
            annotations[current_annotation].append(line)

    for key in annotations:
        annotations[key] = ' '.join(annotations[key])

    for key, value in annotations.items():
        final_annotations.append(value)

    return final_annotations

In [73]:
!pip install rouge

In [74]:
import os
from rouge import Rouge


In [75]:
def calculate_rouge_scores(hypotheses, references):
    rouge = Rouge()
    scores = rouge.get_scores(hypotheses, references, avg=True)
    return scores

In [76]:
data_dir = '/kaggle/working/AESLC/enron_subject_line/test'

In [84]:
def extract_text_subject(email_body):
    
    pattern_with_ann0 = r'@subject\n(.+?)\n@ann0'
    pattern_without_ann0 = r'@subject\n(.+)'
    if re.search('@ann0', email_body):
        match = re.search(pattern_with_ann0, email_body, re.DOTALL)
        if match:
            return match.group(1).strip()
    else:
        match = re.search(pattern_without_ann0, email_body, re.DOTALL)
        if match:
            return match.group(1).strip()

    return None 

In [85]:
# Function to load data from a directory
def load_data_from_directory(data_dir):
    predictions = []
    references = []

    # Iterate over files in the directory
    count =1
    for filename in os.listdir(data_dir):
        count = count+1
        if count > 600:
            break
        print(filename)
        with open(os.path.join(data_dir, filename), 'r', encoding='utf-8', errors='ignore') as file:
            email_text = file.read()

            annotations =  extract_annotations(email_text)
       

            subject = extract_text_subject(email_text)
            annotations.append(subject)

            pattern = r'(.+)(?=@subject)'

            
            match = re.search(pattern, email_text, re.DOTALL)
            text_before_subject = ""
            if match:
                text_before_subject = match.group(1).strip()
              
            else:
                print("No match found.")
            text_before_subject = text_before_subject[:512]
            predictions.append(get_words_split(generate_subject(model,tokenizer,text_before_subject)))
            references.append(annotations)
            #print(annotations)




    return predictions, references

In [86]:

predictions, references = load_data_from_directory(data_dir)
predictions = [predictions.strip() for predictions in predictions]
references = [' '.join(reference).strip() for reference in references]

sanders-r_sent_303.subject
taylor-m_inbox_68.subject
sanders-r_inbox_55.subject
shackleton-s_sent_1225.subject
dasovich-j_inbox_1473.subject
donohoe-t_inbox_315.subject
gilbertsmith-d_inbox_44.subject
salisbury-h_inbox_769.subject
zufferli-j_inbox_41.subject
ybarbo-p_inbox_332.subject
buy-r_inbox_724.subject
mann-k_sent_1092.subject
tycholiz-b_inbox_102.subject
scott-s_sent_31.subject
mann-k_sent_2136.subject
ybarbo-p_inbox_111.subject
holst-k_inbox_117.subject
kaminski-v_sent_4141.subject
salisbury-h_inbox_73.subject
beck-s_sent_1020.subject
skilling-j_sent_105.subject
carson-m_inbox_103.subject
jones-t_sent_1391.subject
buy-r_inbox_732.subject
white-s_inbox_346.subject
jones-t_inbox_1036.subject
taylor-m_sent_939.subject
lay-k_inbox_1090.subject
mclaughlin-e_sent_46.subject
dasovich-j_inbox_304.subject
stclair-c_sent_116.subject
perlingiere-d_sent_2068.subject
mims-thurston-p_inbox_66.subject
causholli-m_inbox_32.subject
wolfe-j_inbox_58.subject
salisbury-h_inbox_1044.subject
shackle

In [99]:
predictions

['linda thanks',
 'EROONLINE Cluster Prod proceeded',
 'Incendendo Energy LLC',
 'Brokerage Agreement',
 'WPtf 1 motion to',
 'JSON WILLIAMS - Veronica',
 'TXU Energy Trading Company',
 'EAsington Church Ascension -',
 'Enron Canada Power Due',
 'dan superheater stb boiler',
 'Audio Video Conference thanks',
 'ESA thanks',
 'Employees Wil',
 "I'm not sure if",
 '"Jason pork love"',
 'Telex inmarsat b telex',
 'Iceland',
 'FRED Lagrasta Group',
 'Monday, Jan. 23 Mt',
 'Sally',
 'Gorge',
 'Call to Trade Press',
 'ISDA Master Agreement Type',
 'Id prefer Delegation',
 'Included',
 'Mailbox - approaching Maximum',
 'Isda since 500 left.',
 'john brindle - former',
 'Total Mark Market Value',
 'Bid farewell seemly fashion',
 'bob assignment termination agreement',
 'Attention foregoing',
 'Soo woke up in',
 'Video Conference thanks',
 '5639809 - Receive Package',
 'Survey of Ho',
 'ETC & Sonoco',
 'Please note company number',
 'A total of 4',
 'Counterparty Trade - go',
 "I'm a sara.",
 'N

In [89]:
!pip install evaluate
import evaluate
rouge = evaluate.load('rouge')

In [57]:
from datasets import load_metric
rouge = load_metric("rouge")
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Generate predictions and calculate ROUGE scores
all_rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

# Rouge expects input as list of strings for predictions and references
for i in range(len(references)):
    for lbl in references[i]:
        #rouge_output = rouge.compute(predictions=preds, references=lbl, use_stemmer=True)
        scores = scorer.score(hypotheses[i], lbl)
        for key in all_rouge_scores.keys():
          all_rouge_scores[key].append(scores[key].fmeasure)


avg_rouge_scores = {key: sum(scores)/len(scores) for key, scores in all_rouge_scores.items()}
print("Average ROUGE Scores:", avg_rouge_scores)

/tmp/ipykernel_36/114105089.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Average ROUGE Scores: {'rouge1': 0.00026454017067928306, 'rouge2': 0.0, 'rougeL': 0.00026454017067928306}


In [49]:
result111 = rouge.compute(predictions=hypotheses, references=references)

NameError: name 'rouge' is not defined

In [65]:
result = rouge.compute(predictions=predictions, references=references)
results_sacrebleu = sacrebleu.compute(predictions=predictions, references=references, lowercase = True)

# print(result)
print(results_sacrebleu)
print("R1:", round(result["rouge1"], 4))
print("R2:", round(result["rouge2"], 4))
print("RL:", round(result["rougeL"], 4))
print("RLsum:", round(result["rougeLsum"], 4))
print("bleu :", round(results_sacrebleu["score"], 4))

{'score': 0.1004185335918131, 'counts': [879, 229, 48, 8], 'totals': [2396, 1797, 1198, 599], 'precisions': [36.686143572621035, 12.743461324429605, 4.006677796327212, 1.335559265442404], 'bp': 0.014198897120310708, 'sys_len': 2396, 'ref_len': 12590}
R1: 0.1294
R2: 0.0386
RL: 0.1235
RLsum: 0.124
bleu : 0.1004


In [62]:
!pip install sacrebleu

sacrebleu = evaluate.load("sacrebleu")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 891.1 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 1.9 MB/s eta 0:00:00 0:00:01
